# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [4]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [5]:
players_180_190 = player_data[(player_data['height'] >= 180) & (player_data['height'] <= 190)].shape[0]

In [6]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [7]:
player_data['birthday'] = pd.to_datetime(player_data['birthday'])
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154


In [8]:
players_1980 = int(player_data['birthday'].apply(lambda x: x.year == 1980).sum())

In [9]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [10]:
highest_players = player_data.sort_values(by=['weight', 'player_name'], ascending=[False, True])['player_name']\
                    .head(10).values.tolist()

In [11]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [12]:
year_b = player_data[player_data['birthday'] \
            .apply(lambda x: (x.year >= 1980) and (x.year <= 1990))] \
            .groupby(player_data['birthday'].map(lambda x: x.year)).count()['id']

In [13]:
years_born_players = list(zip(year_b.index, year_b))

In [14]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [15]:
player_data['player_name'].str.split().str[0]

0           Aaron
1           Aaron
2           Aaron
3           Aaron
4           Aaron
           ...   
11055     Zoumana
11056       Zsolt
11057       Zsolt
11058       Zurab
11059    Zvjezdan
Name: player_name, Length: 11060, dtype: object

In [16]:
adrians = player_data[player_data['player_name'].str.split().str[0] == 'Adriano']['height']

In [17]:
adriano_mean, adriano_std = adrians.mean(), adrians.std()


In [18]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [19]:
dow_with_min_players_born = player_data.groupby(player_data['birthday']\
                                                .map(lambda x: x.day_name()))['id'].count().idxmin()
dow_with_min_players_born

'Sunday'

In [20]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [21]:
matches_data = pd.read_sql("SELECT * FROM Match;", db)
matches_data.head(1)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.4,5.0,1.75,3.35,4.2,1.85,3.2,3.5,1.8,3.3,3.75,NaN,NaN,NaN,1.7,3.3,4.33,1.9,3.3,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


In [22]:
league_data = pd.read_sql("SELECT * FROM League;", db)
league_data.head(1)

,id,country_id,name
0,1,1,Belgium Jupiler League


In [23]:
leagues_agg = matches_data.merge(league_data, how='outer', left_on='league_id', right_on='id')\
                .groupby('name')\
                .count().sort_values(by='id_x', ascending=False)


In [24]:
league_most_matches = leagues_agg[leagues_agg['id_x'] == leagues_agg['id_x'].max()].sort_index().head(1).index[0]

In [25]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [26]:
players_match_ind = ['home_player_%s' % i for i in range(1, 12)] + ['away_player_%s' % i for i in range(1, 12)]
match_players = matches_data[players_match_ind]
ls = []
for match in match_players.values.tolist():
    for player in match:
        ls.append(player)
player_most_matches = pd.Series([x for x in ls if np.isnan(x) == False]).value_counts(ascending=False).idxmax()
player_data[player_data['player_api_id'] == player_most_matches]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
9993,10009,31293,Steve Mandanda,163705,1985-03-28,185.42,181


In [27]:
max_matches_player = player_data[player_data['player_api_id'] == player_most_matches].player_name.values[0]
max_matches_player

'Steve Mandanda'

In [28]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [29]:
player_attributes =  pd.read_sql("SELECT * FROM Player_Attributes;", db)

In [30]:
player_attrs = player_attributes.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)
# player_attrs.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).index

corr_matrix = player_attrs.corr().abs()



sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False)).head(5)


<ipython-input-30-14b25feaa244>:8: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [31]:
top_correlated_features = sol.index.tolist()
top_correlated_features

[('gk_positioning', 'gk_reflexes'),
 ('gk_handling', 'gk_reflexes'),
 ('gk_handling', 'gk_positioning'),
 ('standing_tackle', 'sliding_tackle'),
 ('marking', 'standing_tackle')]

In [32]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [78]:
neymar_fifa_id = player_data[player_data.player_name == 'Neymar']
neymar_fifa_id

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
7855,7867,19533,Neymar,190871,1992-02-05,175.26,150


In [79]:
players_unique = player_attributes.sort_values('date').drop_duplicates('player_fifa_api_id', keep='last')
players_skills = players_unique.drop(['id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)

neymar_skills = players_skills[players_skills.player_fifa_api_id==190871]
neymar_skills = neymar_skills.iloc[0, :]
neymar_skills[0]

190871.0

In [80]:
players_skills = players_skills.fillna(0)

In [81]:
players_skills

,player_fifa_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
144773,174009,56.0,70.0,34.0,56.0,55.0,42.0,0.0,45.0,0.0,32.0,40.0,53.0,53.0,67.0,0.0,43.0,0.0,56.0,0.0,52.0,56.0,47.0,55.0,52.0,36.0,0.0,56.0,23.0,23.0,0.0,18.0,23.0,40.0,23.0,23.0
80570,150550,66.0,74.0,25.0,25.0,25.0,23.0,0.0,25.0,0.0,9.0,65.0,21.0,64.0,26.0,0.0,52.0,0.0,34.0,0.0,39.0,53.0,25.0,54.0,71.0,63.0,0.0,73.0,25.0,25.0,0.0,63.0,67.0,65.0,64.0,73.0
115891,8038,68.0,76.0,69.0,56.0,37.0,74.0,0.0,71.0,0.0,47.0,73.0,70.0,78.0,68.0,0.0,43.0,0.0,62.0,0.0,74.0,74.0,43.0,68.0,66.0,65.0,0.0,74.0,47.0,44.0,0.0,7.0,22.0,73.0,22.0,22.0
13937,148853,59.0,64.0,33.0,31.0,62.0,55.0,0.0,54.0,0.0,34.0,47.0,43.0,64.0,64.0,0.0,59.0,0.0,44.0,0.0,64.0,68.0,31.0,54.0,51.0,54.0,0.0,58.0,59.0,62.0,0.0,6.0,24.0,47.0,24.0,24.0
140697,152782,67.0,69.0,72.0,61.0,63.0,63.0,0.0,66.0,0.0,65.0,59.0,69.0,71.0,69.0,0.0,59.0,0.0,65.0,0.0,66.0,66.0,66.0,61.0,64.0,69.0,0.0,62.0,70.0,67.0,0.0,7.0,25.0,59.0,25.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57229,197948,73.0,77.0,51.0,42.0,75.0,68.0,42.0,47.0,47.0,41.0,67.0,62.0,49.0,55.0,52.0,67.0,47.0,77.0,55.0,74.0,92.0,66.0,77.0,72.0,46.0,32.0,43.0,74.0,75.0,69.0,11.0,15.0,15.0,12.0,7.0
1329,231638,72.0,84.0,49.0,78.0,71.0,63.0,56.0,74.0,45.0,43.0,49.0,73.0,82.0,81.0,60.0,71.0,60.0,73.0,61.0,64.0,63.0,70.0,35.0,20.0,69.0,59.0,77.0,20.0,20.0,21.0,6.0,14.0,13.0,8.0,6.0
153508,172555,68.0,70.0,33.0,70.0,56.0,63.0,60.0,63.0,55.0,27.0,56.0,66.0,90.0,84.0,73.0,60.0,64.0,75.0,67.0,63.0,59.0,69.0,83.0,37.0,75.0,45.0,66.0,23.0,17.0,23.0,13.0,11.0,6.0,9.0,14.0
19675,166074,72.0,75.0,67.0,67.0,57.0,66.0,69.0,80.0,67.0,73.0,58.0,77.0,78.0,90.0,68.0,73.0,55.0,85.0,65.0,78.0,83.0,70.0,56.0,24.0,67.0,61.0,66.0,19.0,24.0,21.0,15.0,7.0,8.0,7.0,11.0


In [86]:
sims = []
for player in players_skills.values:
    dist = []
    for i in range(1, len(player)):
        
        if np.isnan(player[i]) == False:
            
            dist.append(neymar_skills[i] - player[i])
    dist = np.linalg.norm(dist)
    sims.append((player[0], dist))
simular = sorted(sims, key=lambda x: x[1])


ans = []
for i, j in simular:
    if player_data[player_data.player_fifa_api_id == i].shape[0] > 0:
        ans.append(player_data[player_data.player_fifa_api_id == i].player_name.values[0])
    if len(ans) == 6:
        break
ans

['Neymar',
 'Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

In [87]:
players_skills[players_skills['player_fifa_api_id'] == 200529]

,player_fifa_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
129797,200529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
# ans = ans[::-1]
ans

['Neymar',
 'Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

In [92]:
neymar_similarities = ans[1:]

In [90]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [41]:
teams_data = pd.read_sql("SELECT * FROM Team;", db)

In [42]:
borussia_team_api = teams_data[teams_data.team_long_name == 'Borussia Dortmund'].team_api_id.values[0]
germ_1_league = league_data[league_data.name == 'Germany 1. Bundesliga'].id.values[0]

In [43]:
borussia_bundesliga_2008_2009_matches = matches_data[((matches_data['home_team_api_id'] == borussia_team_api)) & 
             (matches_data['season'] == '2008/2009') & \
             (matches_data['league_id'] == germ_1_league)].shape[0]

In [44]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [45]:
from functools import reduce
teams_played = matches_data[(matches_data['season'] == '2008/2009') & \
             (matches_data['league_id'] == germ_1_league)][['home_team_api_id', 'away_team_api_id']]
matches = pd.Series(reduce(lambda x,y: x+y, teams_played.values.tolist())) \
            .value_counts().sort_values(ascending=False).values[0]

In [46]:
team_most_matches_bundesliga_2008_2009 = int(matches)

In [47]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [48]:
matches_data['date'] = pd.to_datetime(matches_data['date'])

In [49]:
arsenal_team_api = teams_data[teams_data.team_long_name == 'Arsenal'].team_api_id.values[0]
arsenal_won_matches = matches_data[(matches_data['season'] == '2015/2016')]

In [50]:
arsenal_won_matches = arsenal_won_matches[((arsenal_won_matches['home_team_api_id'] == arsenal_team_api) & \
                      (arsenal_won_matches['home_team_goal'] > arsenal_won_matches['away_team_goal'])) | \
                     ((arsenal_won_matches['away_team_api_id'] == arsenal_team_api) & \
                      (arsenal_won_matches['home_team_goal'] < arsenal_won_matches['away_team_goal']))]

In [51]:
arsenal_won_matches.shape

(20, 115)

In [52]:
arsenal_won_matches_2015_2016 = arsenal_won_matches.shape[0]

In [53]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [54]:
matches_2015_2016 = matches_data[matches_data['season'] == '2015/2016']
n_matches_2015_2016 = matches_2015_2016.shape[0]
teams_api_id = set(matches_2015_2016['home_team_api_id'].values.tolist() + matches_2015_2016['away_team_api_id'].values.tolist())

In [55]:
def calculate_win_rate(teams):
    win_rate = []
    for team in teams:
        wins_num = matches_2015_2016[((matches_2015_2016['home_team_api_id'] == team) & \
                      (matches_2015_2016['home_team_goal'] > matches_2015_2016['away_team_goal'])) | \
                     ((matches_2015_2016['away_team_api_id'] == team) & \
                      (matches_2015_2016['home_team_goal'] < matches_2015_2016['away_team_goal'])) & \
                     (matches_2015_2016['season'] == '2015/2016')].shape[0]
        team_games_played = matches_2015_2016[(matches_2015_2016['home_team_api_id'] == team) | (matches_2015_2016['away_team_api_id'] == team)].shape[0]
        win_rate.append((team, wins_num/team_games_played))
    win_rate_sorted = sorted(win_rate, key=lambda x: x[1], reverse=True)
    return win_rate_sorted

In [56]:
best_team = calculate_win_rate(teams_api_id)[0]
best_team

(9772, 0.8529411764705882)

In [57]:
best_team_row = teams_data[teams_data['team_api_id'] == best_team[0]]
best_team_row

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
216,35294,9772,234.0,SL Benfica,BEN


In [58]:
team_highest_winrate_2015_2016 = best_team_row['team_long_name'].values[0]

In [59]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [60]:
eng_league_id = league_data[league_data['name'] == 'England Premier League'].id.values[0]
eng_2010_2011 = matches_data[(matches_data['league_id']==eng_league_id) & (matches_data['season'] == '2010/2011')]
eng_2010_2011 = eng_2010_2011[['date', 'home_team_api_id', 'away_team_api_id']]
eng_teams_2010 = set(eng_2010_2011['home_team_api_id'].values.tolist() + eng_2010_2011['home_team_api_id'].values.tolist())

In [61]:
team_big_gap = []
for team in eng_teams_2010:
    team_games = eng_2010_2011[(eng_2010_2011['home_team_api_id'] == team) | (eng_2010_2011['away_team_api_id'] == team)]
    dates = team_games.sort_values(by='date')['date']
    max_gap = dates.diff()[1:].dt.days.values.max()
    team_big_gap.append((team, max_gap))

team_big_gap = sorted(team_big_gap, key=lambda x: x[1], reverse=True)
best_teams = []
max_gap = team_big_gap[0][1]
for i in range(len(team_big_gap)):
    if team_big_gap[i][1] != max_gap:
        break
    best_teams.append(team_big_gap[i][0])

best_teams

[8650, 8658]

In [62]:
bested_teams = teams_data[teams_data['team_api_id'].isin(best_teams)].sort_values(by='team_long_name')
bested_teams

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
45,4218,8658,88.0,Birmingham City,BIR
30,3462,8650,9.0,Liverpool,LIV


In [63]:
highest_gap_england_2010_2011 = int(max_gap)

In [64]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [65]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")